# Setup Flow

This notebook guides you through the setup flow of a new project.

### Variables
Set project environment. Default environment is `dev`.

In [3]:
env = "dev"

Define basic project values.

In [4]:
# Note: This value is hard-coded in /environments/{env}/versions.tf
bucket_name = "nest-vite-fs-microservices-terraform-state-dev"

project_id = "nest-vite-fs-microservices"
project_name = "Nest Vite FS Microservices"

region = "us-central1"

db_tier = "db-f1-micro"

## Database Credentials

In [5]:
auth_db_user="auth_user"
auth_db_password="auth_password"

events_db_user="events_user"
events_db_password="events_password"

Create `tfvars` file for the `environments` folder.

In [5]:
import os

# Path to the terraform.tfvars file
tfvars_file_path = f"environments/{env}/terraform.tfvars"

# Check if the file exists
if os.path.isfile(tfvars_file_path):
  # If the file exists, delete it
  os.remove(tfvars_file_path)
  print(f"Existing terraform.tfvars file deleted: {tfvars_file_path}")

# Variables for the terraform.tfvars file content
terraform_tfvars_content = f"""environment = "{env}"
project_id = "{project_id}"
region = "{region}"

db_tier= "{db_tier}"
"""

# Write content to the terraform.tfvars file
with open(tfvars_file_path, "w") as f:
  f.write(terraform_tfvars_content)

print(f"terraform.tfvars file created at: {tfvars_file_path}")

terraform.tfvars file created at: environments/dev/terraform.tfvars


## Change CHDIR

We need to change the current working directory to the `environments/{env}` folder so that terraform commands can be executed in the correct directory.

In [6]:
import os
os.chdir(f"./environments/{env}")
print("Current working directory:", os.getcwd())

Current working directory: /Users/michaljarnot/IdeaProjects/nest-microservices-example-v2/infra/environments/dev


## Google Cloud Project Setup

In [7]:
# Authenticate with Google Cloud
!gcloud auth login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=32555940559.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fappengine.admin+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcompute+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=9gxLgnAZYetCFx5tmz3ZKgHOnNLZxH&access_type=offline&code_challenge=8UmCRT1KVpZ0mWOGNwxcp96JsTY95QK82mCjRNAtbFY&code_challenge_method=S256


You are now logged in as [m.jarnot@yahoo.com].
Your current project is [nest-vite-fs-microservices].  You can change this setting by running:
  $ gcloud config set project PROJECT_ID


In [8]:
# Create a new Google Cloud project
!gcloud projects create {project_id} --name="{project_name}"

ERROR: (gcloud.projects.create) Project creation failed. The project ID you specified is already in use by another project. Please try an alternative ID.


In [8]:
# Set your Google Cloud project
!gcloud config set project {project_id}

Updated property [core/project].


In [10]:
# Set the default region and zone for your project
!gcloud auth application-default login

Your browser has been opened to visit:

    https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=764086051850-6qr4p6gpi6hn506pt8ejuq83di341hur.apps.googleusercontent.com&redirect_uri=http%3A%2F%2Flocalhost%3A8085%2F&scope=openid+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fuserinfo.email+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fcloud-platform+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fsqlservice.login+https%3A%2F%2Fwww.googleapis.com%2Fauth%2Faccounts.reauth&state=CQxKVdmopQk79dxK7G5XxUmuloiQcS&access_type=offline&code_challenge=PLgBFC02QQEG-KWZsAuxQ5Ul7DWYhr2am4V1_swtNMc&code_challenge_method=S256


Credentials saved to file: [/Users/michaljarnot/.config/gcloud/application_default_credentials.json]

These credentials will be used by any library that requests Application Default Credentials (ADC).

Quota project "nest-vite-fs-microservices" was added to ADC which can be used by Google client libraries for billing and quota. Note that some services may still bill the proj

## Required Google Cloud Services
Link billing account to your project.

In [11]:
import webbrowser
billing_account_link = "https://console.cloud.google.com/billing/linkedaccount?hl=en&project=" + project_id
webbrowser.open(billing_account_link)

True

In [12]:
!gcloud services enable \
  artifactregistry.googleapis.com \
  compute.googleapis.com \
  cloudbuild.googleapis.com \
  secretmanager.googleapis.com \
  sqladmin.googleapis.com \
  storage.googleapis.com \
  iam.googleapis.com \
  cloudresourcemanager.googleapis.com \
  run.googleapis.com \
  vpcaccess.googleapis.com \
  servicenetworking.googleapis.com \
  cloudapis.googleapis.com \
  containerregistry.googleapis.com

Operation "operations/acf.p2-821837556923-b8ad902b-0126-4f1b-b8de-3317ed510d2e" finished successfully.


## GCS Bucket for Terraform State:

In [15]:
# Create a GCS Bucket
!gsutil mb -l {region} gs://{bucket_name}/

Creating gs://nest-vite-fs-microservices-terraform-state-dev/...


In [16]:
# Enable Versioning
!gsutil versioning set on gs://{bucket_name}/

Enabling versioning for gs://nest-vite-fs-microservices-terraform-state-dev/...


In [ ]:
## Secrets Manager

In [28]:
# Database Credentials Secrets

# auth
!gcloud secrets create {env}-auth-db-user --replication-policy="automatic"
!echo -n {auth_db_user} | gcloud secrets versions add {env}-auth-db-user --data-file=-

!gcloud secrets create {env}-auth-db-password --replication-policy="automatic"
!echo -n {auth_db_password} | gcloud secrets versions add {env}-auth-db-password --data-file=-

# events
!gcloud secrets create {env}-events-db-user --replication-policy="automatic"
!echo -n {events_db_user} | gcloud secrets versions add {env}-events-db-user --data-file=-

!gcloud secrets create {env}-events-db-password --replication-policy="automatic"
!echo -n {events_db_password} | gcloud secrets versions add {env}-events-db-password --data-file=-

Created secret [dev-auth-db-user].
Created version [1] of the secret [dev-auth-db-user].
Created secret [dev-auth-db-password].
Created version [1] of the secret [dev-auth-db-password].
Created secret [dev-events-db-user].
Created version [1] of the secret [dev-events-db-user].
Created secret [dev-events-db-password].
Created version [1] of the secret [dev-events-db-password].


## Terraform Deployment:

Initialize and apply your Terraform configuration as usual. Include the path to `tfvars`.

**Initialize Terraform**

In [197]:
# Initialize Terraform
!terraform init


Initializing the backend...
Initializing modules...

Initializing provider plugins...
- Reusing previous version of hashicorp/google from the dependency lock file
- Using previously-installed hashicorp/google v5.19.0

Terraform has been successfully initialized!

You may now begin working with Terraform. Try running "terraform plan" to see
any changes that are required for your infrastructure. All Terraform commands
should now work.

If you ever set or change modules or backend configuration for Terraform,
rerun this command to reinitialize your working directory. If you forget, other
commands will detect it and remind you to do so if necessary.


**Apply Terraform changes**

Note: The following command will fail with an error because we haven't uploaded Docker image yet. `│ Error: Error waiting to create Service: resource is in failed state "Ready:False", message: Revision 'some-project-id-546757-dev-service-00001-6sm' is not ready and cannot serve traffic. Image 'us-central1-docker.pkg.dev/some-project-id-546757/some-project-id-546757-dev-repo/app-service:latest' not found.`

In [198]:
!terraform plan

Acquiring state lock. This may take a few moments...
module.cloud_sql.data.google_secret_manager_secret_version.db_usernames["events"]: Reading...
module.cloud_sql.data.google_compute_default_service_account.default: Reading...
module.vpc.google_compute_network.vpc: Refreshing state... [id=projects/nest-vite-fs-microservices/global/networks/nest-vite-fs-microservices-vpc]
module.rabbitmq.google_compute_address.external_ip: Refreshing state... [id=projects/nest-vite-fs-microservices/regions/us-central1/addresses/dev-rabbitmq-external-ip]
module.cloud_run_auth.google_service_account.cloud_run_sa: Refreshing state... [id=projects/nest-vite-fs-microservices/serviceAccounts/dev-auth-service-sa@nest-vite-fs-microservices.iam.gserviceaccount.com]
module.ci_cd_service_account.google_service_account.ci_cd: Refreshing state... [id=projects/nest-vite-fs-microservices/serviceAccounts/dev-ci-cd-service-account@nest-vite-fs-microservices.iam.gserviceaccount.com]
module.cloud_sql.data.google_secret_m

In [204]:
# Execute Terraform apply directly with hardcoded app_service_count and dynamic tfvars_file_path
!terraform apply --auto-approve

Acquiring state lock. This may take a few moments...
module.cloud_sql.data.google_compute_default_service_account.default: Reading...
module.rabbitmq.google_service_account.rabbitmq_sa: Refreshing state... [id=projects/nest-vite-fs-microservices/serviceAccounts/rabbitmq-sa-001@nest-vite-fs-microservices.iam.gserviceaccount.com]
module.cloud_run_events.google_service_account.cloud_run_sa: Refreshing state... [id=projects/nest-vite-fs-microservices/serviceAccounts/dev-events-service-sa@nest-vite-fs-microservices.iam.gserviceaccount.com]
module.rabbitmq.data.google_compute_default_service_account.default: Reading...
module.ci_cd_service_account.google_secret_manager_secret.ci_cd_key_secret: Refreshing state... [id=projects/nest-vite-fs-microservices/secrets/dev-gcp-auth-sa-key]
module.cloud_run_auth.google_service_account.cloud_run_sa: Refreshing state... [id=projects/nest-vite-fs-microservices/serviceAccounts/dev-auth-service-sa@nest-vite-fs-microservices.iam.gserviceaccount.com]
module.

## Docker Image Build & Deployment

In [4]:
!gcloud auth configure-docker


{
  "credHelpers": {
    "asia.gcr.io": "gcloud",
    "eu.gcr.io": "gcloud",
    "gcr.io": "gcloud",
    "marketplace.gcr.io": "gcloud",
    "staging-k8s.gcr.io": "gcloud",
    "us-central1-docker.pkg.dev": "gcloud",
    "us.gcr.io": "gcloud"
  }
}
Adding credentials for all GCR repositories.
gcloud credential helpers already registered correctly.


In [205]:
service_name = "events-service"
app_name = "events-service"
service_folder = "service-events"

In [206]:
# Build the Docker image using the Dockerfile located in the parent directory.
!docker build --no-cache -t {region}-docker.pkg.dev/{project_id}/{project_id}-{env}-repo/{env}-{app_name}:latest -f ../../../apps/{service_folder}/Dockerfile ../../../

[+] Building 0.0s (0/0)  docker:desktop-linux
[+] Building 0.0s (0/1)                                    docker:desktop-linux
[+] Building 0.2s (2/4)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.95kB                                     0.0s
 => [internal] load .dockerignore                                          0.0s
 => => transferring context: 486B                                          0.0s
 => [internal] load metadata for docker.io/library/node:18.20.3-alpine3.1  0.2s
 => [internal] load metadata for docker.io/library/node:18.20.3            0.2s
[+] Building 0.3s (2/4)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.0s
 => => transferring dockerfile: 1.95kB                                     0.0s
 => [internal] load .dockerignore                                         

In [207]:
# Push Docker Image to Artifact Registry
!docker push {region}-docker.pkg.dev/{project_id}/{project_id}-{env}-repo/{env}-{app_name}:latest

The push refers to repository [us-central1-docker.pkg.dev/nest-vite-fs-microservices/nest-vite-fs-microservices-dev-repo/dev-events-service]

75256d1e: Preparing 
830c38d0: Preparing 
317ea137: Preparing 
4f0e0d60: Preparing 
116fada2: Preparing 
5967b068: Preparing 
17dbd802: Preparing 
e9012a3a: Preparing 
2f1fda6d: Preparing 
a7d078a0: Preparing 
ccf8ba19: Preparing 
ce7ef8cf: Preparing 
17dbd802: Waiting g 
17dbd802: Pushed   280.7MB/244.3MBng     512Blatest: digest: sha256:748f75f6ad9f1783b32cca5f893e0ea6aedf36296de61f9ce83363d336f91c94 size: 3241


In [208]:
# Deploy to Google Cloud Run
# NOTE THERE IS HARD_CODED SERVICE NAME BECAUSE FOLDER NAME IS DIFFERENT FROM SERVICE NAME
!gcloud run deploy {project_id}-{env}-{service_name} --image {region}-docker.pkg.dev/{project_id}/{project_id}-{env}-repo/{env}-{app_name}:latest --platform managed --region={region} --allow-unauthenticated

Deploying container to Cloud Run service [nest-vite-fs-microservices-dev-events-service] in project [nest-vite-fs-microservices] region [us-central1]
Deploying...                                                                   
  . Creating Revision...                                                       
  . Routing traffic...                                                         
  . Setting IAM Policy...                                                      
  Deploying...                                                                 



⠛ Deploying...                                                                 



⠹ Deploying...                                                                 



⠼ Deploying...                                                                 



⠶ Deploying...                                                                 



⠧ Deploying...                                                                 



⠏ Deploying...                  

## Secrets for CI Authorization

In [26]:
# Define your variables
service_account_name = f"{env}-ci-cd-service-account"
service_account_email = f"{service_account_name}@{project_id}.iam.gserviceaccount.com"
key_filename = f"{env}-ci-cd-service-account-key.json"

# Construct the gcloud command
gcloud_command = f"gcloud iam service-accounts keys create ../../../{key_filename} --iam-account {service_account_email}"

# Execute the command
!{gcloud_command}

created key [1af2b81d7c7fa9add606cfe5e2766dc29621cdf9] of type [json] as [../../../dev-ci-cd-service-account-key.json] for [dev-ci-cd-service-account@template-nestjs-api.iam.gserviceaccount.com]


## GitHub Secrets

Go to your GitHub repository and navigate to `Settings` > `Secrets`.

Add a new secrets:

- GCP_PROJECT_ID
- GCP_REGION
- GCP_{env}_SA_KEY
- GCP_{env}_REPOSITORY
- GCP_{env}_IMAGE_NAME

In [27]:
# Print secrets for GitHub
print(f"GCP_PROJECT_ID: {project_id}")
print(f"GCP_REGION: {region}")
print(f"GCP_{env.upper()}_SA_KEY: >>contents of {key_filename}<<")
print(f"GCP_{env.upper()}_REPOSITORY: {project_id}-{env}-repo")
print(f"GCP_{env.upper()}_IMAGE_NAME: {env}-{docker_image_name}")

GCP_PROJECT_ID: template-nestjs-api
GCP_REGION: us-central1
GCP_DEV_SA_KEY: >>contents of dev-ci-cd-service-account-key.json<<
GCP_DEV_REPOSITORY: template-nestjs-api-dev-repo
GCP_DEV_IMAGE_NAME: dev-app-service
